# Set up

# Notebook for uploading record to vector database

In [ ]:
!pip install pinecone-client
!pip install langchain
!pip install beautifulsoup4
!pip install tiktoken
!pip install openai
!pip install python-dotenv

In [3]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.base import Embeddings
from langchain.vectorstores.base import VectorStore
import pinecone
import os
import uuid
from typing import Any, Iterable, List, Optional, Type
from langchain.docstore.document import Document
from loguru import logger
from langchain.vectorstores.base import VectorStoreRetriever

c:\Users\yueli\anaconda3\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
from langchain.document_loaders import ReadTheDocsLoader
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
import os
import sys
sys.path.append("..")

In [ ]:
from apps.slackbot import config as cfg

# openai_api_key = cfg.OPENAI_API_KEY

# PINECONE

In [2]:
from dotenv import load_dotenv

load_dotenv()
load_dotenv(dotenv_path=".env")

import os

PINECONE_INDEX = os.environ.get("PINECONE_INDEX")
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
PINECONE_ENV = os.environ.get("PINECONE_ENV")
openai_api_key = os.environ.get("openai_api_key")
test = os.environ.get("test")

In [3]:
openai_api_key

'api_key'

In [ ]:
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)
active_indexes = pinecone.list_indexes()
index = pinecone.Index("langchain")
index.describe_index_stats()

# Read the doc

### File

In [59]:
"""Loader that loads ReadTheDocs documentation directory dump."""
from pathlib import Path
from typing import Any, List, Optional, Tuple, Union

from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader

# this loader should be used if the original loader from langchain return 0 page content
class ReadTheDocsLoader_custom(BaseLoader):
    """Loader that loads ReadTheDocs documentation directory dump."""

    def __init__(
        self,
        path: Union[str, Path],
        encoding: Optional[str] = None,
        errors: Optional[str] = None,
        custom_html_tag: Optional[Tuple[str, dict]] = None,
        **kwargs: Optional[Any]
    ):
        """
        Initialize ReadTheDocsLoader

        The loader loops over all files under `path` and extract the actual content of
        the files by retrieving main html tags. Default main html tags include
        `<main id="main-content>`, <`div role="main>`, and `<article role="main">`. You
        can also define your own html tags by passing custom_html_tag, e.g.
        `("div", "class=main")`. The loader iterates html tags with the order of
        custom html tags (if exists) and default html tags. If any of the tags is not
        empty, the loop will break and retrieve the content out of that tag.

        Args:
            path: The location of pulled readthedocs folder.
            encoding: The encoding with which to open the documents.
            errors: Specifies how encoding and decoding errors are to be handled—this
                cannot be used in binary mode.
            custom_html_tag: Optional custom html tag to retrieve the content from
                files.
        """
        print("new loader")
        try:
            from bs4 import BeautifulSoup
        except ImportError:
            raise ImportError(
                "Could not import python packages. "
                "Please install it with `pip install beautifulsoup4`. "
            )

        try:
            _ = BeautifulSoup(
                "<html><body>Parser builder library test.</body></html>", **kwargs
            )
        except Exception as e:
            raise ValueError("Parsing kwargs do not appear valid") from e

        self.file_path = Path(path)
        self.encoding = encoding
        self.errors = errors
        self.custom_html_tag = custom_html_tag
        self.bs_kwargs = kwargs

    def load(self) -> List[Document]:
        """Load documents."""
        docs = []
        for p in self.file_path.rglob("*"):
            if p.is_dir():
                continue
            with open(p, encoding=self.encoding, errors=self.errors) as f:
                text = self._clean_data(f.read())
            metadata = {"source": str(p)}
            docs.append(Document(page_content=text, metadata=metadata))
        return docs

    def _clean_data(self, data: str) -> str:
        from bs4 import BeautifulSoup

        soup = BeautifulSoup(data, **self.bs_kwargs)

        # default tags
        html_tags = [
            ("div", {"role": "main"}),
            ("main", {"id": "main-content"}),
        ]
        for para in soup.find_all("p"):
          print(para.get_text())


        if self.custom_html_tag is not None:
            html_tags.append(self.custom_html_tag)

        text = ""
        for para in soup.find_all("p"):
          if para.get_text() is not None:
            text = text + para.get_text()

        # reversed order. check the custom one first
        # for tag, attrs in html_tags[::-1]:
        #     text = soup.find(tag, attrs)
        #     # if found, break
        #     if text is not None:
        #         break

        # if text is not None:
        #     text = text.get_text()
        # else:
        #     text = ""
        # trim empty lines
        return "\n".join([t for t in text.split("\n") if t])


### Get docs

In [ ]:

# Fixie https://docs.fixie.ai/                    y
## LangChain https://python.langchain.com/en/latest/        y
## GPT index https://gpt-index.readthedocs.io/en/latest/     y
## GPTCache https://gptcache.readthedocs.io/en/latest/       y
## GPT4ALL https://docs.gpt4all.io/                 y

# Auto GPT https://docs.agpt.co/                 y
# db-GPT https://db-gpt.readthedocs.io/en/latest/         y
# # AgentGPT https://docs.reworkd.ai/                 ???

## YOLOX https://yolox.readthedocs.io/
## https://textattack.readthedocs.io/en/master/
## https://cowrie.readthedocs.io
## https://pixellib.readthedocs.io/en/latest/

## https://django-storages.readthedocs.io/en/latest/
## https://msal-python.readthedocs.io/en/latest/

In [129]:
# !wget -r -A.html -P rtdocs  Fixie https://docs.fixie.ai/         # Y
# !wget -r -A.html -P rtdocs  LangChain https://python.langchain.com/en/latest/   # y
# !wget -r -A.html -P rtdocs  GPT index https://gpt-index.readthedocs.io/en/latest/    # y custom
# !wget -r -A.html -P rtdocs  GPTCache https://gptcache.readthedocs.io/en/latest/     # y
# !wget -r -A.html -P rtdocs  GPT4ALL https://docs.gpt4all.io/              # y custom

!wget -r -A.html -P rtdocs  Auto GPT https://docs.agpt.co/             # y
!wget -r -A.html -P rtdocs  db-GPT https://db-gpt.readthedocs.io/en/latest/       #  y custom
!wget -r -A.html -P rtdocs  AgentGPT https://docs.reworkd.ai/               #

--2023-06-13 09:30:54--  http://auto/
Resolving auto (auto)... failed: No address associated with hostname.
wget: unable to resolve host address ‘auto’
--2023-06-13 09:30:54--  http://gpt/
Resolving gpt (gpt)... failed: Name or service not known.
wget: unable to resolve host address ‘gpt’
--2023-06-13 09:30:54--  https://docs.agpt.co/
Resolving docs.agpt.co (docs.agpt.co)... 34.148.97.127, 34.74.170.74, 2600:1f18:2489:8202::c8, ...
Connecting to docs.agpt.co (docs.agpt.co)|34.148.97.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9391 (9.2K) [text/html]
Saving to: ‘rtdocs/docs.agpt.co/index.html’

docs.agpt.co/index. 100%[===================>]   9.17K  --.-KB/s    in 0s      

2023-06-13 09:30:55 (87.5 MB/s) - ‘rtdocs/docs.agpt.co/index.html’ saved [9391/9391]

Loading robots.txt; please ignore errors.
--2023-06-13 09:30:55--  https://docs.agpt.co/robots.txt
Reusing existing connection to docs.agpt.co:443.
HTTP request sent, awaiting response... 404 Not Fo

In [155]:
custom = True
file_path = "/content/rtdocs/docs.reworkd.ai"
if custom:
  loader = ReadTheDocsLoader_custom(file_path, features='html.parser')
else:
  loader = ReadTheDocsLoader(file_path, features='html.parser')

new loader


In [156]:
documents = loader.load()

Welcome to AgentGPT 👋, an autonomous AI Agent platform that empowers users to create and deploy customizable autonomous AI agents directly in the web.
Simply assign a name and goal to your AI agent, and watch as it embarks on an exciting journey to accomplish the assigned objective.
AgentGPT works by chaining language models (Agents) to perform a given goal.
Recursively, an agent will think of the best tasks to perform a goal, execute on those tasks, evaluate how it performed, and continually think of more tasks.
We think the power of AI should be available to everyone and should be driven by community.
This is why we are proudly open source. We'd love to hear your feedback at every step of the journey and the following channels are open for you to reach the team:
The entire AgentGPT team is incredibly excited for the road to follow.
We have a ton of exciting features planned for the future, and we hope you'll join us on this journey.
Have a glimpse of what's to come in the roadmap on 

In [157]:
documents[0]

Document(lc_kwargs={'page_content': "Welcome to AgentGPT 👋, an autonomous AI Agent platform that empowers users to create and deploy customizable autonomous AI agents directly in the web.\nSimply assign a name and goal to your AI agent, and watch as it embarks on an exciting journey to accomplish the assigned objective.AgentGPT works by chaining language models (Agents) to perform a given goal.\nRecursively, an agent will think of the best tasks to perform a goal, execute on those tasks, evaluate how it performed, and continually think of more tasks.We think the power of AI should be available to everyone and should be driven by community.\nThis is why we are proudly open source. We'd love to hear your feedback at every step of the journey and the following channels are open for you to reach the team:The entire AgentGPT team is incredibly excited for the road to follow.\nWe have a ton of exciting features planned for the future, and we hope you'll join us on this journey.\nHave a glimp

In [158]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

In [159]:

text_splitter = CharacterTextSplitter(separator ="\n", chunk_size=2000, chunk_overlap=100)  # need to modify separator for different documents
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [ ]:
docs[2]

In [160]:
for doc in docs:
  if len(doc.page_content) == 0:
    docs.remove(doc)

len(docs)

1

In [149]:
texts = []
meta = []

for doc in docs:
  texts.append(doc.page_content)

  metadata = {}
  source = doc.metadata['source'].replace("/content/rtdocs/","https://")  # may not "https://".
  metadata["source"] = source
  meta.append(metadata)

In [83]:
meta[0]

{'source': 'python.langchain.com/en/latest/search.html'}

# VectorStore

In [62]:
from loguru import logger
import os
import uuid
from typing import Any, Iterable, List, Optional, Tuple, Type

import pinecone
from langchain.docstore.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.base import Embeddings
# from langchain.indexes import VectorstoreIndexCreator
# from langchain.text_splitter import CharacterTextSplitter
# from langchain.vectorstores import Chroma
from langchain.vectorstores.base import VectorStore, VectorStoreRetriever

# import config as cfg
# from utils import load_files



class ConversationStore(VectorStore):
    def __init__(self, namespace, db, embeddings, text_key):
        self.db = db
        self.namespace = namespace
        self.embeddings = embeddings
        self.text_key = text_key

    @classmethod
    def from_index(cls, namespace, openai_api_key, index_name, text_key="text"):
        pinecone.init(api_key=cfg.PINECONE_API_KEY, environment=cfg.PINECONE_ENV)
        logger.info(f"Loading index {index_name} from Pinecone")
        index = pinecone.Index(index_name)
        embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
        return cls(namespace, index, embeddings, text_key)

    def add_text(self, text: str, metadata={}) -> str:
        metadata[self.text_key] = text
        id = str(uuid.uuid4())
        embedding = self.embeddings.embed_query(text)
        doc = {"id": id, "values": embedding, "metadata": metadata}
        self.db.upsert(vectors=[doc], namespace=self.namespace)

        return id

    def add_texts(self, texts: Iterable[str], metadatas: List[dict]) -> List[str]:
        for text, metadata in zip(texts, metadatas):
            self.add_text(text, metadata)

    def similarity_search(
        self,
        text: str,
        top_k: int = 5,
        filter: Optional[dict] = None,
        threshold: float = 0.7,
    ) -> list[Document]:
        query_embedding = self.embeddings.embed_query(text)
        results = self.db.query(
            [query_embedding],
            top_k=top_k,
            include_metadata=True,
            namespace=self.namespace,
            filter=filter,
        )

        docs = []
        for res in results["matches"]:
            metadata = res["metadata"]
            text = metadata.pop(self.text_key)
            if res["score"] > threshold:
                docs.append(Document(page_content=text, metadata=metadata))
        return docs

    def _similarity_search_with_relevance_scores(
        self,
        query: str,
        k: int = 4,
        **kwargs: Any,
    ) -> List[Tuple[Document, float]]:
        logger.debug("query", query)
        query_embedding = self.embeddings.embed_query(query)
        results = self.db.query(
            [query_embedding],
            top_k=k,
            include_metadata=True,
            namespace=self.namespace,
            filter=kwargs.get("filter", None),
        )

        docs_with_score = []
        for res in results["matches"]:
            metadata = res["metadata"]
            text = metadata.pop(self.text_key)
            docs_with_score.append(
                (Document(page_content=text, metadata=metadata), res["score"])
            )
        logger.debug(docs_with_score)
        return docs_with_score

    @classmethod
    def delete(cls, namespace, index_name):
        pinecone.init(api_key=cfg.PINECONE_API_KEY, environment=cfg.PINECONE_ENV)
        index = pinecone.Index(index_name)
        return index.delete(delete_all=True, namespace=namespace)

    @classmethod
    def get_vector_retrieval(
        cls,
        namespace: str,
        openai_api_key: str,
        index_name: str,
        search_type="similarity",
        search_kwargs={},
    ) -> VectorStoreRetriever:
        vectorstore = cls.from_index(namespace, openai_api_key, index_name)
        retriever = VectorStoreRetriever(
            vectorstore=vectorstore,
            search_type=search_type,
            search_kwargs=search_kwargs,
        )
        return retriever

    @classmethod
    def from_texts(cls, texts: List[str], embedding: Embeddings, metadatas: list[dict]):
        raise NotImplementedError("ConversationStore does not support from_texts")


# class LocalChromaStore(Chroma):
#     @classmethod
#     def from_folder(cls, file_path, openai_api_key, index_name="chroma"):
#         """
#         Create a Chroma DB from a folder of files (Currently only supports pdfs and
#         markdown files)
#         file_path: path to the folder
#         openai_api_key: openai api key
#         index_name: name of the index
#         """
#         files = os.listdir(file_path)
#         files = [file_path + "/" + file for file in files]
#         documents = load_files(files)

#         embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
#         chroma = cls(index_name, embeddings)
#         test_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
#         documents = test_splitter.split_documents(documents)
#         logger.info("adding documents")
#         chroma.add_documents(documents)
#         return chroma

# Upload docs

In [ ]:
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)
active_indexes = pinecone.list_indexes()
index = pinecone.Index("langchain")
index.describe_index_stats()

In [ ]:
# if you can  not use functions in the ConversationStore, you can simply use these one.
def add_texts(texts: Iterable[str], meta: List[dict]):
    for text, metadata in zip(texts, meta):
            source = metadata["source"]
            metadata = {"text": text, "source": source}
            id = str(uuid.uuid4())
            embedding = embeddings.embed_query(text)
            doc = {"id": id, "values": embedding, "metadata": metadata}

            index.upsert(vectors=[doc], namespace="ReadTheDocs")


In [63]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectorstore = ConversationStore("ReadTheDocs", index, embeddings, 'text')

AttributeError: can't set attribute

In [162]:
vectorstore.db.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'01662450-28d4-49b7-8484-5f27f34a2cdc': {'vector_count': 24},
                '023e8e97-eea9-4538-9fd7-afad634087df': {'vector_count': 3},
                '06596c06-638b-4dd2-954f-acd5e21c8231': {'vector_count': 1},
                '06e6ba9b-a273-4148-99c0-1613b5f9f239': {'vector_count': 2},
                '0fc4f143-60b0-44b3-9865-1e85db59299c': {'vector_count': 1},
                '11873f72-c580-4fea-83fe-e4bbe72f0410': {'vector_count': 1},
                '1204cb97-7bf6-4617-8897-8355fa8da366': {'vector_count': 1},
                '16f0978b-32e5-4115-90e2-09690d718331': {'vector_count': 92},
                '175a95ed-6317-40e6-88ed-095f4d5dc03c': {'vector_count': 6},
                '195bff2d-2aa4-40e3-9a03-443347da8f61': {'vector_count': 7},
                '19cd51a5-e33c-4f5f-bca9-3bca31391006': {'vector_count': 30},
                '1cf26fb0-89f1-4db8-a26a-45fe1e29244a': {'vector_count': 18},
                '2538d125-43

In [161]:
vectorstore.add_texts(texts, meta)

In [164]:
d = vectorstore.similarity_search("what is langchain")
# result = ""
# for doc in docs:
#     result += doc.page_content + "\n\n"

# print(result)

d[0].metadata

{'source': 'python.langchain.com/en/latest/modules/chains/examples/flare.html'}